**Random Forest: 2009-2013 training set**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from datetime import datetime
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

The data was provided and it contains stock market data fo approximately 1500 securities listed on the US stock exchange. We'll work with daily data from February 2018 to July 2017. 

In [2]:
data=pd.read_csv("./final_project_data.csv")
data=data.dropna()
data.index=range(len(data))

In [13]:
rf= RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample")
print(rf.get_params())

{'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': 99, 'verbose': 0, 'warm_start': False}


In [12]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 45, stop = 105, num = 15)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 8, num = 5)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

#Grid
random_grid = {'n_estimators': n_estimators,'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [10]:
#Experiment for the first training set: 2008-2012
i=2013

train_rf_2=data[(data.date<=int(str(i)+'1231'))&(data.date>=int(str(i-4)+'0101'))]
train_rf_2=train_rf_2.dropna()
X_train_rf_2=np.asarray(train_rf_2.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_train_rf_2=np.asarray(train_rf_2.loc[:,'target'])

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 5, verbose=2, random_state=42, n_jobs = -1, pre_dispatch=500)
rf_random.fit(X_train_rf_2, Y_train_rf_2)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed: 60.3min remaining: 98.3min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed: 135.9min remaining: 15.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 141.7min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=99, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
          pre_dispatch=500, random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [14]:
rf_random.best_params_

{'n_estimators': 49,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 5}

Checking accuracy

In [16]:
test_rf_2=data[(data.date<=int(str(i+1)+'1231'))&(data.date>=int(str(i+1)+'0101'))]
test_rf_2=test_rf_2.dropna()
X_test_rf_2=np.asarray(test_rf_2.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_test_rf_2=np.asarray(test_rf_2.loc[:,'target'])

acc_os=1.0*(rf_random.best_estimator_.predict(X_test_rf_2)==np.asarray(Y_test_rf_2)).sum()/len(Y_test_rf_2)
acc_in=1.0*(rf_random.best_estimator_.predict(X_train_rf_2)==np.asarray(Y_train_rf_2)).sum()/len(Y_train_rf_2)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 49.97706126947061
In sample accuracy: 52.349439499919455


In [19]:
X_train_,X_vali_,Y_train_,Y_vali_ = train_test_split(X_train_rf_2, Y_train_rf_2, test_size=0.1, random_state=99)

best_score = 0
n_estimators = [int(x) for x in np.linspace(start = 45, stop = 60, num = 7)]
max_depth = [int(x) for x in np.linspace(4, 8, num = 3)]
for n in n_estimators:
    for d in max_depth:
        clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators= n,
        min_samples_split= 5, min_samples_leaf= 2, max_features='sqrt', max_depth=d, n_jobs=-1)
        clf.fit(X_train_,Y_train_) 
        score = clf.score(X_vali_, Y_vali_)
        if score > best_score:
            best_score = score
        print ("Score:", score, "n_estimator= ", n, "Max_depth=",d )

Score: 0.5170782974251182 n_estimator=  45 Max_depth= 4
Score: 0.5212422033859581 n_estimator=  45 Max_depth= 6
Score: 0.5238696337590533 n_estimator=  45 Max_depth= 8
Score: 0.5165356759350225 n_estimator=  47 Max_depth= 4
Score: 0.521185085334369 n_estimator=  47 Max_depth= 6
Score: 0.5236240261372204 n_estimator=  47 Max_depth= 8
Score: 0.5168326898032854 n_estimator=  50 Max_depth= 4
Score: 0.5212079325550046 n_estimator=  50 Max_depth= 6
Score: 0.5241609358221573 n_estimator=  50 Max_depth= 8
Score: 0.5169811967374169 n_estimator=  52 Max_depth= 4
Score: 0.5211736617240512 n_estimator=  52 Max_depth= 6
Score: 0.5242808837304942 n_estimator=  52 Max_depth= 8
Score: 0.5170611620096415 n_estimator=  55 Max_depth= 4
Score: 0.5213678630994539 n_estimator=  55 Max_depth= 6
Score: 0.5240866823550915 n_estimator=  55 Max_depth= 8
Score: 0.5169811967374169 n_estimator=  57 Max_depth= 4
Score: 0.521190797139528 n_estimator=  57 Max_depth= 6
Score: 0.5244979323265325 n_estimator=  57 Max_dep

In [21]:
clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators= 50,
min_samples_split= 5, min_samples_leaf= 2, max_features='sqrt', max_depth=8, n_jobs=-1)
clf.fit(X_train_rf_2,Y_train_rf_2) 

acc_os=1.0*(clf.predict(X_test_rf_2)==np.asarray(Y_test_rf_2)).sum()/len(X_test_rf_2)
acc_in=1.0*(clf.predict(X_train_rf_2)==np.asarray(Y_train_rf_2)).sum()/len(Y_train_rf_2)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 50.15134707391612
In sample accuracy: 53.71484808294464
